In [6]:
# Kütüphaneler

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import mean_squared_error , r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale     # standardizasoyon işlemleri
from sklearn.preprocessing import StandardScaler    # standardizasoyon işlemleri
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

from warnings import filterwarnings
filterwarnings("ignore")     # önceki bölümlerde çalıştırdığımızda çıkan gereksiz uyarı mesajları çıkmasın diye

In [3]:
df = pd.read_csv("C:\python_py\Makine Öğrenmesi-2\Hitters.csv")
df = df.dropna()

dms = pd.get_dummies(df[["League" , "Division" , "NewLeague"]])

y = df["Salary"]

x__ = df.drop(["Salary" , "League" , "Division" , "NewLeague"] , axis=1).astype("float64")
x = pd.concat([x__ , dms[["League_N" , "Division_W" , "NewLeague_N"]]] , axis=1)

x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.25 , random_state=42)

# MODEL

In [13]:
svr_model = SVR(kernel="linear").fit(x_train , y_train)     # kernel değerini değiştirebiliriz örneğin "rbf" yaptığımızda bu sefer hata oranımız artarak 470'e geliyor.

# TAHMİN

In [15]:
svr_model.predict(x_train)[0:10]

array([ 219.32622627,  702.43039317,  623.20559641,  153.77538484,
        463.15191157, 1002.96905384,   55.20051607,  218.47517427,
        466.00937227,  802.1635624 ])

In [16]:
svr_model.predict(x_test)[0:10]

array([679.14754919, 633.72883529, 925.68639938, 270.28464317,
       530.26659421, 272.22606175, 549.4423224 , 446.55264396,
       892.8309562 , 677.96856008])

In [17]:
# Doğrunun denklemini çıkartmak istersek

svr_model.intercept_

array([-80.15196063])

In [18]:
svr_model.coef_

array([[ -1.2183904 ,   6.09602978,  -3.67574533,   0.14217072,
          0.51435925,   1.28388992,  12.55922527,  -0.08693754,
          0.46597185,   2.98259931,   0.52944513,  -0.79820793,
         -0.16015531,   0.30872795,   0.28842348,  -1.79560066,
          6.41868986, -10.74313785,   1.33374319]])

In [20]:
# Test setinin ilk hata oranı

y_pred = svr_model.predict(x_test)

RMSE = np.sqrt(mean_squared_error(y_pred , y_test))
RMSE

370.0408415795005

# MODEL TUNİNG (DOĞRULAMA)

In [34]:
# Olası bazı C değerlerini verip bunları GridSearchCV yöntemi ile k-katlı çapraz doğrulama yaparak optimum C değerini bulmalıyız.

svr_params = {"C":[0.1 , 0.5 , 1 , 3]}
svr_cv_model =  GridSearchCV(svr_model , svr_params , cv=5).fit(x_train , y_train)

# svr_model_tuned =  GridSearchCV(svr_model , svr_params , cv=5 , verbose = 2 , n_jobs = -1).fit(x_train , y_train)   -> verbose = 2 değerini de eklersek bize raparlayarak bildirir (kaç dakika çalıştı kaç deneme yaptı vs.)
                                                                                                                     #  n_jobs = -1  ile işlemci max güçte çalışır.

In [35]:
# verbose hiperparametresini denedim. Her bir C değerini 5 defa deniyor çünkü cv=5 verdik

svr_cv_model =  GridSearchCV(svr_model , svr_params , cv=5 , verbose=2).fit(x_train , y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ..............................................C=0.1; total time=   0.2s
[CV] END ..............................................C=0.1; total time=   0.3s
[CV] END ..............................................C=0.1; total time=   0.2s
[CV] END ..............................................C=0.1; total time=   0.2s
[CV] END ..............................................C=0.1; total time=   0.2s
[CV] END ..............................................C=0.5; total time=   1.6s
[CV] END ..............................................C=0.5; total time=   1.3s
[CV] END ..............................................C=0.5; total time=   1.6s
[CV] END ..............................................C=0.5; total time=   1.3s
[CV] END ..............................................C=0.5; total time=   1.5s
[CV] END ................................................C=1; total time=   3.5s
[CV] END ........................................

In [36]:
svr_cv_model.best_params_    # Optimum C değerini bize verdi

{'C': 0.5}

In [37]:
# FİNAL MODELİMİZ

svr_model_tuned = SVR(kernel="linear" , C=0.5).fit(x_train , y_train)

In [38]:
# FİNAL MODEL'imizin hata oranı 

y_pred = svr_model_tuned.predict(x_test)

RMSE = np.sqrt(mean_squared_error(y_pred , y_test))
RMSE

# İlk hata oranımız 370'ti.

367.98747616655294